In [1]:
import networkx as nx
import pandas as pd
from tqdm import tqdm as pbar
import numpy as np
from itertools import islice
import string
from scipy.sparse import lil_matrix, csc_matrix
from itertools import combinations
from tqdm import tqdm_notebook as timer
from scipy.misc import comb
import re
from collections import defaultdict

Download files from the link (contain 6 txt files). Merge actor_movies.txt
and actress_movies.txt files into one file, and remove all actors/actresses
with less than 5 (so actors who have acted in four or fewer number of movies)
movies; Note that you will have to parse the data in these lists as accurately as
possible to extract the entities consistently and create the network. So plan on
spending some time in cleaning the data set.

In [2]:
class AutoIncrement():
    def __init__(self):
        self.count = -1
    
    def next(self):
        self.count += 1
        return self.count
    
    def __str__(self):
        return str(self.count)
    
class IntMap():
    def __init__(self):
        self.data = {}
        self.ai = AutoIncrement()
    
    def decode(self, s):
        if s in self.data:
            return(self.data[s])
        else:
            count = self.ai.next()
            self.data[s] = count
            self.data[count] = s
            return(count)
    
    def encode(self, i):
        if i in self.data:
            return self.data[i]
        else:
            raise KeyError("Cannot encode number " + str(i))
    
    def __str__(self):
        return "Integer Map with %s entries" % (str(self.ai))

In [3]:
mdict = IntMap()
adict = IntMap()
A = {}
M = {}

In [4]:
printable = set(string.printable)

def clean_string(s):
    return(re.sub(r'\(.*\)|\{.*\}|\'|\"', "", s).lstrip().rstrip())

In [5]:
lc = {"data/actress_movies.txt" : 1182813, "data/actor_movies.txt" : 2167653}

In [8]:
for fname in ["data/actor_movies.txt", "data/actress_movies.txt"]:
    with open(fname, "r") as f:
        for line in timer(f, total = lc[fname], desc=fname, ncols=100):
            if line.count('\t\t') < 5:
                pass
            else:
                line = filter(lambda x : x in printable, line.decode('latin1')).encode('ascii')
                splits = line.split("\t\t")
                actor_name =  adict.decode(splits[0])
                movies =  set(map(lambda x : mdict.decode(clean_string(x)), splits[1:]))
                A[actor_name] = movies

                for movie in movies:
                    if movie not in M:
                        M[movie] = [actor_name]
                    else:
                        M[movie].append(actor_name)

In [9]:
num_actors = adict.ai.count + 1
print "Number of actors", str(num_actors)

Number of actors 244269


In [10]:
num_movies = mdict.ai.count + 1
print "Number of movies", str(num_movies)

Number of movies 447439


In [11]:
len(M.keys())

447439

In [12]:
with open('elist.txt', 'w') as f:
    for a in timer(sorted(A.keys()), desc="Writing to edgelist", ncols=30):
        edges = set()
        
        for movie in A[a]:
            for b in M[movie]:
                edges.add(b)

        f.write(''.join(map(lambda x : ' '.join((str(a), str(x), '\n')), sorted(edges))))

In [13]:
with open('actors.txt', 'w') as f:
    for i in timer(range(num_actors), desc="Actor Names", ncols=30):
        f.write("%s\n" % adict.encode(i))

In [14]:
with open('movies.txt', 'w') as f:
    for i in timer(range(num_movies), desc="Movie Names", ncols=30):
        f.write("%s\n" % mdict.encode(i))

In [15]:
import igraph